![Digital Futures](https://github.com/digital-futures-academy/DataScienceMasterResources/blob/main/Resources/datascience-notebook-header.png?raw=true)

## Learner Stories

```text
As a DATA PROFESSIONAL,  
I want to be able to write subqueries and use Common Table Expressions (CTEs),  
so that I can break down complex queries into simpler, reusable components
```

# What are Common Table Expressions (CTEs)?

Common Table Expressions (CTEs) are temporary result sets that you can reference within a SELECT, INSERT, UPDATE, or DELETE statement. They are defined using the WITH keyword and are useful for creating complex queries that are easier to read and maintain.  They are a little bit like context managers in Python, but for SQL!

### Example of a CTE

```sql
WITH cte AS (
    SELECT column1, column2
    FROM table1
    WHERE column3 = 'value'
)
SELECT *
FROM cte;
```

---

## Why Use CTEs?

CTEs are useful for a number of reasons:

1. **Readability**: CTEs can make complex queries easier to read and understand by breaking them down into smaller, more manageable parts.
2. **Reusability**: CTEs can be referenced multiple times within a query, allowing you to reuse the same logic in different parts of the query.
3. **Performance**: CTEs can improve query performance by allowing the database engine to optimize the execution plan.
4. **Debugging**: CTEs can be used to isolate and test specific parts of a query, making it easier to identify and fix issues.
5. **Recursive Queries**: CTEs can be used to create recursive queries that reference themselves, allowing you to work with hierarchical data structures.
6. **Window Functions**: CTEs can be used in conjunction with window functions to perform complex analytical queries. (Window functions are a powerful feature of SQL that allow you to perform calculations across a set of rows related to the current row.)
7. **Data Transformation**: CTEs can be used to transform data in a query, such as aggregating, filtering, or joining data from multiple tables.

---

## Use Cases for CTEs

Here are some common use cases for CTEs:

1. **Recursive Queries**: CTEs can be used to create recursive queries that reference themselves, allowing you to work with hierarchical data structures such as organizational charts, bill of materials, or social networks.
2. **Reusable Queries**: CTEs can be used to define reusable subqueries that can be referenced multiple times within a query, reducing duplication and improving maintainability.
3. **Data Transformation**: CTEs can be used to transform data in a query, such as aggregating, filtering, or joining data from multiple tables.

Here is an example of a ***recursive*** CTE that generates a list of all employees and their managers in an organizational chart:

```sql
WITH RECURSIVE employee_hierarchy AS (
    SELECT employee_id, manager_id
    FROM employees
    WHERE manager_id IS NULL
    UNION ALL
    SELECT e.employee_id, e.manager_id
    FROM employees e
    JOIN employee_hierarchy eh ON e.manager_id = eh.employee_id
)
SELECT *
FROM employee_hierarchy;
```

This query does the following:

- The initial query selects the root node of the hierarchy (employees with no manager).
- The recursive part of the query joins the employees table with the CTE to find the direct reports of each manager.
- The UNION ALL operator combines the results of the initial query and the recursive part to generate the final result set.
- The SELECT statement at the end retrieves the final result set.
- The query will continue to execute recursively until all levels of the hierarchy have been processed.

---

## Demo Database Set Up

To see some examples in action, we'll create an in-memory SQLite database and populate it with some sample data. We'll then run a recursive query to generate an organizational chart.

#### Create and Populate the Database

In [ ]:
import sqlite3

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create the employees table
cursor.execute('''
CREATE TABLE employees (
    employee_id INTEGER PRIMARY KEY,
    name TEXT,
    manager_id INTEGER
)
''')

# Populate the employees table with sample data
employees = [
    (1, 'Alice', None),  # Top-level employee with no manager
    (2, 'Bob', 1),       # Employee under Manager Alice
    (3, 'Charlie', 1),   # Another Employee under Manager Alice
    (4, 'David', 2),     # Employee under Manager Bob
    (5, 'Eve', 2),       # Another Employee under Manager Bob
    (6, 'Frank', 3),     # Employee under Manager Charlie
    (7, 'Grace', 4),     # Employee under Employee David
]

cursor.executemany('INSERT INTO employees (employee_id, name, manager_id) VALUES (?, ?, ?)', employees)

---

## 1. Recursive Query Demo

#### Define the Query as a String variable in Python

In [ ]:
recursive_query = """
WITH RECURSIVE employee_hierarchy AS (
    SELECT employee_id, manager_id
    FROM employees
    WHERE manager_id IS NULL
    UNION ALL
    SELECT e.employee_id, e.manager_id
    FROM employees e
    JOIN employee_hierarchy eh ON e.manager_id = eh.employee_id
)
SELECT *
FROM employee_hierarchy;
"""

#### Run the Query and put the result into a Pandas DataFrame

In [ ]:
# Import the pandas library
import pandas as pd

In [ ]:
# Run the query using the read_sql function
df = pd.read_sql(recursive_query, conn)

# Print the DataFrame
df

#### Why is the `manager_id` column a float?

The issue likely arises from the way the Pandas handling the data types.  By default, pandas may interpret columns with missing values or mixed types as floats. You can explicitly set the data type of the manager_id column to int when creating the DataFrame.

In [ ]:
# Re-run the query using the read_sql function
df = pd.read_sql(recursive_query, conn)

# Explicitly set the data type of the manager_id column to integer
df['manager_id'] = df['manager_id'].astype('Int64')  # Use 'Int64' for nullable integers

# Print the DataFrame
df

#### Why does `employee-id=1` have `<NA>` for the `manager_id`?

The employee with `employee_id=1` has no manager, so the `manager_id` is NULL in the database. When the data is loaded into a Pandas DataFrame, the NULL value is represented as `<NA>`.  You can replace the `<NA>` values with `None` or any other value using the `fillna()` method.  Let's replace the `<NA>` values with `0` for this employee.  It is often better to keep values as `None` or `NULL` in the database.  We often remove `NULL` values from the data when we load it into a DataFrame.  However, in this case, we want to keep the `NULL` values to represent the absence of a manager and therefore, we'll convert the `<NA>` values to `0`.

In [ ]:
df['manager_id'] = df['manager_id'].fillna(0)

# Print the DataFrame
df

---

---

## 2. Reusability Query Demo

CTEs can be referenced multiple times within a query, allowing you to reuse the same logic in different parts of the query.  Let's see an example of this.

#### Define the CTE Query as a String variable in Python

In [ ]:
# Define the CTE query as a String variable in Python

reusable_cte_query = '''
WITH RECURSIVE employee_hierarchy AS (
    SELECT employee_id, name, manager_id, 0 AS depth
    FROM employees
    WHERE manager_id IS NULL
    UNION ALL
    SELECT e.employee_id, e.name, e.manager_id, eh.depth + 1
    FROM employees e
    JOIN employee_hierarchy eh ON e.manager_id = eh.employee_id
)
'''

#### Define and Execute some Queries that re-use the CTE

In [ ]:
# Query 1: Select all employees in the hierarchy
query1 = reusable_cte_query + '''
SELECT *
FROM employee_hierarchy;
'''

# Query 2: Count the number of employees under each manager
query2 = reusable_cte_query + '''
SELECT manager_id, COUNT(employee_id) AS num_employees
FROM employee_hierarchy
GROUP BY manager_id;
'''

# Query 3: Find the depth of the hierarchy for each employee
query3 = reusable_cte_query + '''
SELECT employee_id, name, manager_id, depth 
FROM employee_hierarchy;
'''

#### Execute the Query and put the result into a Pandas DataFrame

In [ ]:
# Dictionary to store DataFrames
dfs = {}

# Execute each query and store the result in the dictionary
for i, query in enumerate([query1, query2, query3], start=1):
    df_name = f'df{i}'
    dfs[df_name] = pd.read_sql(query, conn)
    if 'manager_id' in dfs[df_name].columns:
        dfs[df_name]['manager_id'] = dfs[df_name]['manager_id'].astype('Int64').fillna(0)
    
# Display the DataFrames
for df_name, df in dfs.items():
    print(f"{df_name}:")
    print(df)
    print("\n")

The organisation chart data is generated by the CTE and then used in the subsequent queries to find the employees who report to a specific manager.

We can use `Matplotlib` and another library to visualize the data in the DataFrame.  We can use `networkx` to create a graph of the organization chart and `pydot` to help with its layout.  `networkx` is a Python library for creating, manipulating, and studying complex networks but it will help create the visualisation we want here.  `pydot` is a Python interface to Graphviz's Dot language and it will help with the layout of the graph (you may need to have Graphviz installed on your system to use `pydot`):

> On a Mac, you can install Graphviz using Homebrew with the command `brew install graphviz`.
> On Windows, you can download the Graphviz installer from the Graphviz website and run it to install Graphviz.

In [ ]:
# Import the libraries
import matplotlib.pyplot as plt
import networkx as nx

In [ ]:
# Create a directed graph
G = nx.DiGraph()

In [ ]:
# Specify the dataframe to plot
df = dfs['df3']

df.head()

In [ ]:
# Create a directed graph
G = nx.DiGraph()

# Add nodes and edges
for _, row in df.iterrows():
    G.add_node(row["name"])
    if row["manager_id"] != 0:  # Exclude the root manager
        manager_name = df.loc[df["employee_id"] == row["manager_id"], "name"].values[0]
        G.add_edge(manager_name, row["name"])

# Use pydot layout for hierarchical positioning - note - pydot has to be installed as a dependency! pip install pydot
pos = nx.nx_pydot.pydot_layout(G, prog="dot")  # "dot" ensures hierarchical layout

# Draw the graph
plt.figure(figsize=(10, 6))
nx.draw(
    G, pos,
    with_labels=True,
    node_size=3000,
    node_color="lightblue",
    font_size=10,
    font_color="black",
    edge_color="gray",
    arrowsize=20
)
plt.title("Organization Chart (Minimal Crossing)")
plt.show()

---

---

## 3. Data Transformation Query Demo

CTEs can be used to transform data in a query, such as aggregating, filtering, or joining data from multiple tables.  Let's see an example of this.

### Modify and Add to the Database for the Demo

In [ ]:
# Create additions to the in-memory SQLite database for aggregating and joining data from multiple tables

# Create the departments table

cursor.execute('''
CREATE TABLE departments (
    department_id INTEGER PRIMARY KEY,
    name TEXT
)
''')

# Populate the departments table with sample data

departments = [
    (1, 'Engineering'),
    (2, 'Marketing'),
    (3, 'Sales'),
    (4, 'Executive')
]

cursor.executemany('INSERT INTO departments (department_id, name) VALUES (?, ?)', departments)

# Add department information to the employees table
cursor.execute('''
ALTER TABLE employees
ADD COLUMN department_id INTEGER
''')

# Update the department_id column with sample data
cursor.execute('''
UPDATE employees
SET department_id = 1
WHERE employee_id IN (2, 4, 5, 7)
''')

cursor.execute('''
UPDATE employees
SET department_id = 2
WHERE employee_id = 3
''')

cursor.execute('''
UPDATE employees
SET department_id = 3
WHERE employee_id = 6
''')

cursor.execute('''
UPDATE employees
SET department_id = 4
WHERE employee_id = 1
''')


# Create the salaries table
cursor.execute('''
CREATE TABLE salaries (
    employee_id INTEGER PRIMARY KEY,
    salary INTEGER
)
''')

# Populate the salaries table with sample data

salaries = [
    (1, 100000),
    (2, 80000),
    (3, 75000),
    (4, 70000),
    (5, 60000),
    (6, 65000),
    (7, 55000),
]

cursor.executemany('INSERT INTO salaries (employee_id, salary) VALUES (?, ?)', salaries)


### Query 4 - Data Transformation - Joining

The following query will join the employees table with the salaries and department tables and simplify the data by selecting only the relevant columns.

> ***QUESTIONS***:
> 
> 1. Why may it be desirable to extract the salary data into a separate table?
> 2. Why does this feel like bad database design?
> 3. What features of the database could be used to help the security of the salary data?

In [ ]:
# Define a query to list the employee name, department name, manager name and salary
query4 = '''
SELECT e.name AS employee_name, d.name AS department_name, m.name AS manager_name, s.salary
FROM employees e
JOIN departments d ON e.department_id = d.department_id
LEFT JOIN employees m ON e.manager_id = m.employee_id
JOIN salaries s ON e.employee_id = s.employee_id
'''

In [ ]:
# Execute the query and convert into a DataFrame
df4 = pd.read_sql(query4, conn)

# Display the DataFrame
df4

***Hang on just a minute...that query is not a CTE!  However, is it easy to read?  Could it be improved?  Let's see how we can use a CTE to make this query more readable.***

In [ ]:
employee_details_cte = '''
WITH employee_details AS (
    SELECT 
        e.employee_id,
        e.name AS employee_name,
        e.manager_id,
        e.department_id,
        s.salary
    FROM 
        employees e
    JOIN 
        salaries s ON e.employee_id = s.employee_id
)
'''

department_details_cte = '''
department_details AS (
    SELECT 
        d.department_id,
        d.name AS department_name
    FROM 
        departments d
)
'''

manager_details_cte = '''
manager_details AS (
    SELECT 
        m.employee_id,
        m.name AS manager_name
    FROM 
        employees m
)
'''

actual_query = '''
SELECT 
    ed.employee_name, 
    dd.department_name, 
    md.manager_name, 
    ed.salary
FROM 
    employee_details ed
JOIN 
    department_details dd ON ed.department_id = dd.department_id
LEFT JOIN 
    manager_details md ON ed.manager_id = md.employee_id;
'''

# Define the main query to join the CTEs
query5 = employee_details_cte + ', ' + department_details_cte + ', ' + manager_details_cte + actual_query


In [ ]:
# Execute the query and convert into a DataFrame
df5 = pd.read_sql(query5, conn)

df5

---

---

## Over to you...

### Activity 1

Given the tables already created, write query that aggregates the salary data by department and calculates the average salary for each department.

In [ ]:
# Query to aggregate average salary by department
# You may use the previously defined CTEs for employee and department details if you wish
# Redefine the CTEs if you wish to practice your own!
# Use the same connection object (conn)

query6 = '''

'''

# Run the query and store the result in a DataFrame

df6 = pd.read_sql(query6, conn)

df6

---

### Activity 2

How much are we paying out (in total salary) those who are 2 tiers (`depth = 2`) below the CEO in the hierarchy?

In [ ]:
# Query to find the total salary expense for those who are on depth = 2 of the hierarchy

query7 = '''

'''

# Run the query and store the result in a DataFrame
df7 = pd.read_sql(query7, conn)

df7

---

## Query Solutions

#### Activity 1

In [ ]:
WITH employee_details AS (
    SELECT 
        e.employee_id,
        e.name AS employee_name,
        e.department_id,
        s.salary
    FROM 
        employees e
    JOIN 
        salaries s ON e.employee_id = s.employee_id
),
department_salaries AS (
    SELECT 
        ed.department_id,
        AVG(ed.salary) AS avg_salary
    FROM 
        employee_details ed
    GROUP BY 
        ed.department_id
)
SELECT
    dd.name,
    ds.avg_salary
FROM
    department_salaries ds
JOIN
    departments dd ON ds.department_id = dd.department_id;

#### Activity 2

In [ ]:
WITH RECURSIVE employee_hierarchy AS (
    SELECT employee_id, manager_id, 0 AS depth
    FROM employees
    WHERE manager_id IS NULL
    UNION ALL
    SELECT e.employee_id, e.manager_id, eh.depth + 1
    FROM employees e
    JOIN employee_hierarchy eh ON e.manager_id = eh.employee_id
),
depth_2_employees AS (
    SELECT employee_id
    FROM employee_hierarchy
    WHERE depth = 2
)
SELECT
    SUM(s.salary) AS total_salary_expense
FROM
    salaries s
JOIN
    depth_2_employees d2e ON s.employee_id = d2e.employee_id;